# Solving Quintics

In this section, we explain how to exploit the icosahedral symmetry discussed before to solve an arbitrary polynomial of order five (also known as a quintic polynomial).

We begin by returning to a result derived earlier: in the Section on [icosahedral symmetry](icosa_fc), we showed that the the icosahedral face centres can be grouped into five octohedra:

$$
\begin{aligned}
T(u,v) &= \prod_{k=1}^5 t_I(u\,\epsilon^{-k/2},v\,\epsilon^{k/2}) = \prod_{k=1}^5 \epsilon^{15-3k} \,t_I(u,v\,\epsilon^k) \\
&= u^{30} +522\left(u^{25}\,v^5-u^5\,v^{25}\right) - 10005\left(u^{20}\,v^{10} + u^{10}\,v^{20}\right) + v^{30} \,.
\end{aligned}


$$

where

$$

t_I(u,v) := (u^2+v^2)(u^2-2z_n\,uv-v^2)(u^2-2z_m\,uv-v^2) \,.

$$

In [1]:
# Check of the above:
from utils import e_simp, reduce_multiply
from sympy import symbols, expand, simplify

u, v, e = symbols("u v e")
z_n = e**2 + e**3
z_m = e + e**4


def tI(u, v):
    return (
        (u**2 + v**2)
        * (u**2 - 2 * z_n * u * v - v**2)
        * (u**2 - 2 * z_m * u * v - v**2)
    )


def T(u, v):
    return (
        u**30
        + 522 * (u**25 * v**5 - u**5 * v**25)
        - 10005 * (u**20 * v**10 + u**10 * v**20)
        + v**30
    )


T_eval = e_simp(
    expand(reduce_multiply([e ** (15 - 3 * k) * tI(u, v * e**k) for k in range(5)])), e
)
T_eval.equals(T(u, v))

True

What we intend to do next is to demonstrate the existence of a map from points
$z$ on the complex plane (away from the edge midpoints) to a particular set of
roots of a Brioschi quintic, whose Brioschi parameter can also be computed from
$z$.

To do this, we begin by considering the polynomial $P(X)$ given by

$$

 P(X) = \prod_{k=1}^5 \left(X - \epsilon^{15-3k} \,t_I(u,v\,\epsilon^k)\right) \,.

$$

Expanding, we find that

$$

P(X) = X^5 - 10\,f(u,v)\,X^3+45\,f(u,v)^2\,X - T(u,v)

$$

where $f(u,v) = uv\left(u^{10}+11\,u^5v^5-v^{10}\right)$, as in the previous
[Section](icosa).

In [2]:
# Check of the above:
X = symbols("X")
result = e_simp(
    reduce_multiply([X - e ** (15 - 3 * k) * tI(u, v * e**k) for k in range(5)]), e
).collect(X)
result

KeyboardInterrupt: 

In [ ]:
# Check the term linear in X is 45*f(u,v)^2, specifically
def f(u, v):
    return u * v * (u**10 + 11 * u**5 * v**5 - v**10)


result.coeff(X).equals(45 * f(u, v) ** 2)

True

In [ ]:
# Check the whole polynomial
result.equals(X**5 - 10 * f * X**3 + 45 * f**2 * X - T)

True

Therefore, if we define the homogeneous coordinates

$$

x_k(u,v) := -\frac{f^2(u,v)}{T(u,v)}\,\epsilon^{15-3k}\,t_I(u,v\,\epsilon^k) \,,

$$

(which are degree-zero complex functions in $u$ and $v$, defined everywhere
except for at the edge midpoints) then a second polynomial can be readily
constructed as follows:

$$

P_B(X,z) = \prod_{k=1}^5 \left(X- x_k(z,1)\right) \,.

$$

The polynomial $P_B(X)$ is indeed a Brioschi quintic with Brioschi parameter

$$

B(z) = -\frac{f^5(z,1)}{T^2(z,1)}

$$

such that

$$

P_B(X,z) = X^5 +10\,B(z)\, X^3 + 45\,B^2(z)\,X + B^2(z)

$$

## Tschirnhaus Transformation and Principal Quintics

In this section, we aim to demonstrate that solving a single quadratic equation
allows us to trade any general quintic of the form

$$

P(x) = x^5+ p_1 \,x^4 + p_2 \, x^3 + p_3 \, x^2+p_4 \, x + p_5 = 0

$$

for a principal quintic with $a_1 = a_2 = 0$.

To this end, let's define a transformation 

$$

y = x^2 + A\,x + B

$$

and show how to solve for $(A,B,C)$ such that the polynomial equation $Q(y)=0$
with coefficients $q_i$ with $q_1 = q_2 = 0$.

The polynomial $Q(y)$ after this transformation is given by the __Resolvent__,
$\mathrm{Res}[P(x), y-x^2-Ax-B](y)$:

$$
\begin{aligned}
Q(y) &= y^{5} + y^{4} \left(A p_{1} - 5 B - p_{1}^{2} + 2 p_{2}\right) \\
&+ y^{3} \left(A^{2} p_{2} - 4 A B p_{1} - A p_{1} p_{2} + 3 A p_{3} + 10 B^{2} + 4 B p_{1}^{2} - 8 B p_{2} - 2 p_{1} p_{3} + p_{2}^{2} + 2 p_{4}\right) + \ldots
\end{aligned}
$$

In [ ]:
# Check of the above:
from sympy import resultant, solve, Eq, Poly, Add

x, p1, p2, p3, p4, p5 = symbols("x p1 p2 p3 p4 p5")
expr = x**5 + p1 * x**4 + p2 * x**3 + p3 * x**2 + p4 * x + p5
Eq(expr, 0)

Eq(p1*x**4 + p2*x**3 + p3*x**2 + p4*x + p5 + x**5, 0)

In [ ]:
y, A, B = symbols("y A B")

expr2 = Poly(resultant(expr, y - x**2 - A * x - B, x), y).terms()
expr2 = Add(*[b * y ** a[0] for a, b in expr2], evaluate=False)
expr2

y**5 + y**4*(A*p1 - 5*B - p1**2 + 2*p2) + y**3*(A**2*p2 - 4*A*B*p1 - A*p1*p2 + 3*A*p3 + 10*B**2 + 4*B*p1**2 - 8*B*p2 - 2*p1*p3 + p2**2 + 2*p4) + y**2*(A**3*p3 - 3*A**2*B*p2 - A**2*p1*p3 + 4*A**2*p4 + 6*A*B**2*p1 + 3*A*B*p1*p2 - 9*A*B*p3 - 3*A*p1*p4 + A*p2*p3 + 5*A*p5 - 10*B**3 - 6*B**2*p1**2 + 12*B**2*p2 + 6*B*p1*p3 - 3*B*p2**2 - 6*B*p4 - 2*p1*p5 + 2*p2*p4 - p3**2) + y*(A**4*p4 - 2*A**3*B*p3 - A**3*p1*p4 + 5*A**3*p5 + 3*A**2*B**2*p2 + 2*A**2*B*p1*p3 - 8*A**2*B*p4 - 4*A**2*p1*p5 + A**2*p2*p4 - 4*A*B**3*p1 - 3*A*B**2*p1*p2 + 9*A*B**2*p3 + 6*A*B*p1*p4 - 2*A*B*p2*p3 - 10*A*B*p5 + 3*A*p2*p5 - A*p3*p4 + 5*B**4 + 4*B**3*p1**2 - 8*B**3*p2 - 6*B**2*p1*p3 + 3*B**2*p2**2 + 6*B**2*p4 + 4*B*p1*p5 - 4*B*p2*p4 + 2*B*p3**2 - 2*p3*p5 + p4**2) + (A**5*p5 - A**4*B*p4 - A**4*p1*p5 + A**3*B**2*p3 + A**3*B*p1*p4 - 5*A**3*B*p5 + A**3*p2*p5 - A**2*B**3*p2 - A**2*B**2*p1*p3 + 4*A**2*B**2*p4 + 4*A**2*B*p1*p5 - A**2*B*p2*p4 - A**2*p3*p5 + A*B**4*p1 + A*B**3*p1*p2 - 3*A*B**3*p3 - 3*A*B**2*p1*p4 + A*B**2*p2*p3 + 5

Solving for the coefficients $A$ and $B$, we find

$$

\begin{aligned}
A &= \frac{5 B + p_{1}^{2} - 2 p_{2}}{p_{1}} \\
B &=\frac{p_{1}^{2} p_{2} + 3 p_{1} p_{3} - 4 p_{2}^{2} \pm p_{1} \sqrt{\frac{8 p_{1}^{3} p_{3} - 3 p_{1}^{2} p_{2}^{2} + 16 p_{1}^{2} p_{4} - 38 p_{1} p_{2} p_{3} + 12 p_{2}^{3} - 40 p_{2} p_{4} + 45 p_{3}^{2}}5} }{2\left(2 p_{1}^{2} - 5 p_{2}\right)} \,.
\end{aligned}
$$

In [ ]:
# Check of the above:
y4_coeff = expr2.coeff(y**4)
A_val = solve(y4_coeff, A)[0]
Eq(A, A_val)

Eq(A, (5*B + p1**2 - 2*p2)/p1)

In [ ]:
B_vals = solve(expr2.coeff(y**3).subs(A, A_val), B)
B_vals[0]

(p1**2*p2/2 + 3*p1*p3/2 - p1*sqrt(40*p1**3*p3 - 15*p1**2*p2**2 + 80*p1**2*p4 - 190*p1*p2*p3 + 60*p2**3 - 200*p2*p4 + 225*p3**2)/10 - 2*p2**2)/(2*p1**2 - 5*p2)

In [ ]:
B_vals[1]

(p1**2*p2/2 + 3*p1*p3/2 + p1*sqrt(40*p1**3*p3 - 15*p1**2*p2**2 + 80*p1**2*p4 - 190*p1*p2*p3 + 60*p2**3 - 200*p2*p4 + 225*p3**2)/10 - 2*p2**2)/(2*p1**2 - 5*p2)

Therefore, at the expense of a single square root, we can trade any general
quintic for a principal quintic of the form

$$

Q(y) = y^5 + 5\alpha\, y^2 + 5\beta\, y + \gamma = 0 

$$

for some $\alpha$, $\beta$, $\gamma$ computable in terms of the original $p_i$.
(The factors of 5 are included for later convenience.)

## From Brioschi and Principal Quintics to the Icosahedral Equation



In [ ]:
from sympy import series

z = symbols("z")

series(T(z, 1) ** 3 / (1728 * f(z, 1) ** 5), z)

-1/(1728*z**5) + 1511/1728 - 703127*z**5/1728 + O(z**6)

In [ ]:
(
    40 * p1**3 * p3
    - 15 * p1**2 * p2**2
    + 80 * p1**2 * p4
    - 190 * p1 * p2 * p3
    + 60 * p2**3
    - 200 * p2 * p4
    + 225 * p3**2
).factor()

5*(8*p1**3*p3 - 3*p1**2*p2**2 + 16*p1**2*p4 - 38*p1*p2*p3 + 12*p2**3 - 40*p2*p4 + 45*p3**2)

In [ ]:
print(latex(_))

5 \left(8 p_{1}^{3} p_{3} - 3 p_{1}^{2} p_{2}^{2} + 16 p_{1}^{2} p_{4} - 38 p_{1} p_{2} p_{3} + 12 p_{2}^{3} - 40 p_{2} p_{4} + 45 p_{3}^{2}\right)


In [ ]:
y, A, B, C = symbols("y A B C")

odd_poly = Poly.from_list(odd_coeffs, x**2)
even_poly = Poly.from_list(even_coeffs, x**2)
(x**2 * odd_poly.as_expr() ** 2 - even_poly.as_expr() ** 2).expand().subs(
    x**2, y - B * x - C
).expand().collect(y).subs(C, -(p1**2 - 2 * p2) / 5 - B * x).expand().collect(y)

-4*p_1**10/3125 + 8*p_1**8*p_2/625 - 2*p_1**7*p_3/125 - 27*p_1**6*p_2**2/625 + 2*p_1**6*p_4/125 + 12*p_1**5*p_2*p_3/125 - 2*p_1**5*p_5/25 + 34*p_1**4*p_2**3/625 - 2*p_1**4*p_2*p_4/125 - p_1**4*p_3**2/25 - 24*p_1**3*p_2**2*p_3/125 + 8*p_1**3*p_2*p_5/25 - 4*p_1**2*p_2**4/625 - 16*p_1**2*p_2**2*p_4/125 + 4*p_1**2*p_2*p_3**2/25 - 2*p_1**2*p_3*p_5/5 + p_1**2*p_4**2/5 + 16*p_1*p_2**3*p_3/125 - 8*p_1*p_2**2*p_5/25 - 72*p_2**5/3125 + 24*p_2**3*p_4/125 - 4*p_2**2*p_3**2/25 + 4*p_2*p_3*p_5/5 - 2*p_2*p_4**2/5 - p_5**2 + y**5 + y**3*(-2*p_1**4/5 + 8*p_1**2*p_2/5 - 2*p_1*p_3 - 3*p_2**2/5 + 2*p_4) + y**2*(-4*p_1**6/25 + 24*p_1**4*p_2/25 - 6*p_1**3*p_3/5 - 33*p_1**2*p_2**2/25 + 6*p_1**2*p_4/5 + 12*p_1*p_2*p_3/5 - 2*p_1*p_5 + 2*p_2**3/25 - 2*p_2*p_4/5 - p_3**2) + y*(-3*p_1**8/125 + 24*p_1**6*p_2/125 - 6*p_1**5*p_3/25 - 57*p_1**4*p_2**2/125 + 6*p_1**4*p_4/25 + 24*p_1**3*p_2*p_3/25 - 4*p_1**3*p_5/5 + 36*p_1**2*p_2**3/125 - 4*p_1**2*p_2*p_4/25 - 2*p_1**2*p_3**2/5 - 24*p_1*p_2**2*p_3/25 + 8*p_1*p_2*p_5/5 

In [ ]:
even_poly

Poly(p_1*(x**2)**2 + p_3*(x**2) + p_5, x**2, domain='ZZ[p_1,p_3,p_5]')

In [ ]:
odd_poly

Poly((x**2)**2 + p_2*(x**2) + p_4, x**2, domain='ZZ[p_2,p_4]')

In [ ]:
def tIk(k, u, v, e):
    return e ** (15 - k * 3) * tI(u, v * e**k)


e_simp(sum([tIk(k, u, v, e) for k in range(5)]), e)

0

In [ ]:
from utils import hessian

W = e_simp(hessian(tI(u, v), u, v) / (-400), e)
W

u**8 - u**7*v + 7*u**6*v**2 + 7*u**5*v**3 - 7*u**3*v**5 + 7*u**2*v**6 + u*v**7 + v**8

In [ ]:
def Wk(k, u, v, e):
    W = (
        u**8
        - u**7 * v
        + 7 * u**6 * v**2
        + 7 * u**5 * v**3
        - 7 * u**3 * v**5
        + 7 * u**2 * v**6
        + u * v**7
        + v**8
    )
    return e_simp(e ** (20 - k * 4) * W.subs(v, v * e**k), e)


e_simp(sum([Wk(k, u, v, e) for k in range(5)]), e)

0

In [ ]:
Wk(2, u, v, e)

e**3*v*(u**7 + 7*u**5*v**2 - 7*u**2*v**5 + v**7) + e**2*u**2*(u**6 + u**5*v - 7*u*v**5 - 7*v**6) + e*u*v*(u**6 + 7*u**5*v - 7*u*v**5 + v**6) + u**7*v - 7*u**2*v**6

In [ ]:
e_simp(sum([Wk(k, u, v, e) ** 2 for k in range(5)]), e)

0

In [ ]:
e_simp(sum([Wk(k, u, v, e) * tIk(k, u, v, e) for k in range(5)]), e)

0

In [ ]:
e_simp(sum([Wk(k, u, v, e) ** 2 * tIk(k, u, v, e) for k in range(5)]), e)

0

In [ ]:
e_simp(sum([Wk(k, u, v, e) ** 2 * tIk(k, u, v, e) ** 2 for k in range(5)]), e)

0

In [ ]:
def tiWj(i, j, u, v, e):
    return e_simp(
        sum([tIk(k, u, v, e) ** i * Wk(k, u, v, e) ** j for k in range(5)]), e
    )


def f(u, v):
    return u * v * (u**10 + 11 * u**5 * v**5 - v**10)


simplify(tiWj(0, 3, u, v, e) / f(u, v) ** 2)

120

In [ ]:
j = symbols("j")
g2 = -3 * j / (1728 - j)
g3 = j / (1728 - j)
simplify(1728 * g2**3 / (g2**3 - 27 * g3**2))

j

In [ ]:
W

u**8 - u**7*v + 7*u**6*v**2 + 7*u**5*v**3 - 7*u**3*v**5 + 7*u**2*v**6 + u*v**7 + v**8

In [ ]:
sigma, tau, Y = symbols("sigma tau Y")


def Yk(k, sigma, tau, u, v, e):
    tk_val = tIk(k, u, v, e)
    W_val = Wk(k, u, v, e)
    return e_simp(sigma * W_val + tau * tk_val * W_val, e)


Y_poly = (
    e_simp(reduce_multiply([Y - Yk(k, sigma, tau, u, v, e) for k in range(5)]), e)
    .expand()
    .collect(Y)
)
Y_poly

Y**5 + Y**2*(-40*sigma**3*u**22*v**2 - 880*sigma**3*u**17*v**7 - 4760*sigma**3*u**12*v**12 + 880*sigma**3*u**7*v**17 - 40*sigma**3*u**2*v**22 - 5*sigma**2*tau*u**30 - 2610*sigma**2*tau*u**25*v**5 + 50025*sigma**2*tau*u**20*v**10 + 50025*sigma**2*tau*u**10*v**20 + 2610*sigma**2*tau*u**5*v**25 - 5*sigma**2*tau*v**30 - 360*sigma*tau**2*u**33*v**3 - 11880*sigma*tau**2*u**28*v**8 - 129600*sigma*tau**2*u**23*v**13 - 455400*sigma*tau**2*u**18*v**18 + 129600*sigma*tau**2*u**13*v**23 - 11880*sigma*tau**2*u**8*v**28 + 360*sigma*tau**2*u**3*v**33 - 5*tau**3*u**41*v - 2665*tau**3*u**36*v**6 + 21320*tau**3*u**31*v**11 + 552885*tau**3*u**26*v**16 + 552885*tau**3*u**16*v**26 - 21320*tau**3*u**11*v**31 - 2665*tau**3*u**6*v**36 + 5*tau**3*u*v**41) + Y*(5*sigma**4*u**31*v - 1085*sigma**4*u**26*v**6 - 10075*sigma**4*u**21*v**11 + 29450*sigma**4*u**16*v**16 + 10075*sigma**4*u**11*v**21 - 1085*sigma**4*u**6*v**26 - 5*sigma**4*u*v**31 - 90*sigma**2*tau**2*u**42*v**2 + 18540*sigma**2*tau**2*u**37*v**7 + 3962

In [ ]:
(f(u, v) ** 2).expand()

u**22*v**2 + 22*u**17*v**7 + 119*u**12*v**12 - 22*u**7*v**17 + u**2*v**22

In [ ]:
Y2sigma3 = Y_poly.coeff(Y**2).coeff(sigma**3)
Y2sigma3

-40*u**22*v**2 - 880*u**17*v**7 - 4760*u**12*v**12 + 880*u**7*v**17 - 40*u**2*v**22

In [ ]:
Y2sigma3.equals(-40 * f(u, v) ** 2)

True

In [ ]:
Y2sigma2tau = Y_poly.coeff(Y**2).coeff(sigma**2 * tau)
Y2sigma2tau

-5*u**30 - 2610*u**25*v**5 + 50025*u**20*v**10 + 50025*u**10*v**20 + 2610*u**5*v**25 - 5*v**30

In [ ]:
Y2sigma2tau.equals(-5*)

In [ ]:
[Y - Yk(k, sigma, tau, u, v, e) for k in range(5)]

[Y - e**15*sigma*(u**2 + v**2)*(u**2 - u*v*(2*e**3 + 2*e**2) - v**2)*(u**2 - u*v*(2*e**4 + 2*e) - v**2) - e**15*tau*(u**2 + v**2)*(u**2 - u*v*(2*e**3 + 2*e**2) - v**2)*(u**2 - u*v*(2*e**4 + 2*e) - v**2)*(u**8 - u**7*v + 7*u**6*v**2 + 7*u**5*v**3 - 7*u**3*v**5 + 7*u**2*v**6 + u*v**7 + v**8),
 Y - e**12*sigma*(e**2*v**2 + u**2)*(-e**2*v**2 - e*u*v*(2*e**3 + 2*e**2) + u**2)*(-e**2*v**2 - e*u*v*(2*e**4 + 2*e) + u**2) - e**12*tau*(e**2*v**2 + u**2)*(-e**2*v**2 - e*u*v*(2*e**3 + 2*e**2) + u**2)*(-e**2*v**2 - e*u*v*(2*e**4 + 2*e) + u**2)*(e**3*v**2*(7*u**6 - 7*u**5*v + u*v**5 - v**6) - e**2*v*(u**7 + 7*u**5*v**2 - 7*u**2*v**5 + v**7) - e*(-u**8 + 7*u**5*v**3 + 7*u**3*v**5 + v**8) - 7*u**5*v**3 - v**8),
 Y - e**9*sigma*(e**4*v**2 + u**2)*(-e**4*v**2 - e**2*u*v*(2*e**3 + 2*e**2) + u**2)*(-e**4*v**2 - e**2*u*v*(2*e**4 + 2*e) + u**2) - e**9*tau*(e**4*v**2 + u**2)*(-e**4*v**2 - e**2*u*v*(2*e**3 + 2*e**2) + u**2)*(-e**4*v**2 - e**2*u*v*(2*e**4 + 2*e) + u**2)*(e**3*v*(u**7 + 7*u**5*v**2 - 7*u**2*v**

In [ ]:
tI4 = e_simp(tI(u, v) ** 4, e)
tI4

u**24 + 8*u**23*v + 4*u**22*v**2 - 88*u**21*v**3 - 94*u**20*v**4 + 312*u**19*v**5 + 116*u**18*v**6 + 88*u**17*v**7 + 879*u**16*v**8 - 2608*u**15*v**9 + 904*u**14*v**10 - 2288*u**13*v**11 + 476*u**12*v**12 + 2288*u**11*v**13 + 904*u**10*v**14 + 2608*u**9*v**15 + 879*u**8*v**16 - 88*u**7*v**17 + 116*u**6*v**18 - 312*u**5*v**19 - 94*u**4*v**20 + 88*u**3*v**21 + 4*u**2*v**22 - 8*u*v**23 + v**24

In [ ]:
6248 / 22

284.0

In [ ]:
121 + 135

256

In [ ]:
135 + 121

256

In [ ]:
-1 - Rational(135, 121)

-256/121

In [ ]:
from sympy import Rational


a, b = symbols("a b")

(a * W**3 + b * tI4 + chi**2).expand().subs(a, -1 - b).subs(
    b, Rational(135, 121)
).collect(u)

0

In [ ]:
from sympy import solve


solve(8 * b * v - 3 * v * (-b - 1) - 168 * v / 11, b)

[135/121]

In [ ]:
(chi**2).expand()

121*u**24 - 1848*u**23*v + 5604*u**22*v**2 + 6248*u**21*v**3 + 44946*u**20*v**4 - 4488*u**19*v**5 + 50644*u**18*v**6 + 123288*u**17*v**7 + 224631*u**16*v**8 + 261712*u**15*v**9 - 411576*u**14*v**10 + 486288*u**13*v**11 + 666876*u**12*v**12 - 486288*u**11*v**13 - 411576*u**10*v**14 - 261712*u**9*v**15 + 224631*u**8*v**16 - 123288*u**7*v**17 + 50644*u**6*v**18 + 4488*u**5*v**19 + 44946*u**4*v**20 - 6248*u**3*v**21 + 5604*u**2*v**22 + 1848*u*v**23 + 121*v**24

In [ ]:
tI4a

In [ ]:
from utils import jacobian

chi = e_simp(jacobian(tI(u, v), W, u, v), e) / (-22)
chi

u**12 - 84*u**11*v/11 - 6*u**10*v**2 - 20*u**9*v**3 + 15*u**8*v**4 - 24*u**7*v**5 - 84*u**6*v**6 + 24*u**5*v**7 + 15*u**4*v**8 + 20*u**3*v**9 - 6*u**2*v**10 + 84*u*v**11/11 + v**12

In [ ]:
(-22 * tI(u, v) ** 2).expand()

-352*e**14*u**8*v**4 - 704*e**14*u**6*v**6 - 352*e**14*u**4*v**8 - 704*e**13*u**8*v**4 - 1408*e**13*u**6*v**6 - 704*e**13*u**4*v**8 - 352*e**12*u**8*v**4 - 704*e**12*u**6*v**6 - 352*e**12*u**4*v**8 + 352*e**11*u**9*v**3 - 704*e**11*u**8*v**4 + 352*e**11*u**7*v**5 - 1408*e**11*u**6*v**6 - 352*e**11*u**5*v**7 - 704*e**11*u**4*v**8 - 352*e**11*u**3*v**9 + 704*e**10*u**9*v**3 - 1408*e**10*u**8*v**4 + 704*e**10*u**7*v**5 - 2816*e**10*u**6*v**6 - 704*e**10*u**5*v**7 - 1408*e**10*u**4*v**8 - 704*e**10*u**3*v**9 + 704*e**9*u**9*v**3 - 704*e**9*u**8*v**4 + 704*e**9*u**7*v**5 - 1408*e**9*u**6*v**6 - 704*e**9*u**5*v**7 - 704*e**9*u**4*v**8 - 704*e**9*u**3*v**9 - 88*e**8*u**10*v**2 + 1056*e**8*u**9*v**3 - 352*e**8*u**8*v**4 + 1056*e**8*u**7*v**5 - 528*e**8*u**6*v**6 - 1056*e**8*u**5*v**7 - 352*e**8*u**4*v**8 - 1056*e**8*u**3*v**9 - 88*e**8*u**2*v**10 - 352*e**7*u**10*v**2 + 1056*e**7*u**9*v**3 - 704*e**7*u**8*v**4 + 1056*e**7*u**7*v**5 - 704*e**7*u**6*v**6 - 1056*e**7*u**5*v**7 - 704*e**7*u**4*v**

In [ ]:
from sympy import discriminant, symbols

Y, a, b, c = symbols("Y a b c")

Y_poly = Y**5 + 5 * a * Y**2 + 5 * b * Y + c

discriminant(Y_poly, Y)

108*a**5*c - 27*a**4*b**2 + 2250*a**2*b*c**2 - 1600*a*b**3*c + 256*b**5 + 3125*c**4